In [7]:
import pandas as pd
import numpy as np
from pathlib import Path
import yaml
import dask.array as da
from ag3 import release_data
v3_release = release_data()

In [8]:
#samplesets
samplesets = v3_release.all_wild_sample_sets
df_meta = v3_release.load_sample_set_metadata(samplesets, True)
#fixes
df_meta.country = df_meta.country.str.replace("Gambia, The", "The Gambia").str.replace("Equatorial Guinea", "Bioko")
#consolidate all Mayotte sites into one (so we get a pie chart)
df_meta.loc[(df_meta.country == 'Mayotte'),'latitude'] = df_meta[df_meta.country == 'Mayotte'].latitude.mean()
df_meta.loc[(df_meta.country == 'Mayotte'),'longitude'] = df_meta[df_meta.country == 'Mayotte'].longitude.mean()
#counts by sites
df_site_counts = df_meta.groupby(['latitude', 'longitude', 'country']).size().reset_index(name='counts')
df_site_counts['site'] = range(len(df_site_counts))
#>= 10 sites 
df_10 = df_site_counts[df_site_counts.counts >= 10].copy()
len(df_10),len(df_10.latitude.unique()),len(df_10.longitude.unique())

/opt/conda/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


(43, 43, 42)

In [9]:
#new full meta but only containing >=10 sites (to preserve all species columns)
df_10_meta = df_meta.merge(df_10[['longitude','latitude', 'site']])
len(df_10_meta.latitude.unique()),len(df_10_meta.longitude.unique())

(43, 42)

In [11]:
arab_call = pd.crosstab(df_10_meta.site, df_10_meta.species_gambcolu_arabiensis)
colu_call = pd.crosstab(df_10_meta.site, df_10_meta.species_gambiae_coluzzii)
colu_call.rename({"intermediate": "gambiae/coluzzii"}, axis=1, inplace=True)

In [17]:
sex_call = pd.crosstab(df_10_meta.site, df_10_meta.sex_call)
sex_call.rename({"F": "female", "M": "male", "UKN": "sex_unknown"}, axis=1, inplace=True)
sex_call.head()

sex_call,female,male,sex_unknown
site,,,
0,74,0,0
1,41,0,0
2,11,12,0
3,77,4,0
4,43,0,0


# whats with sex_unknown?

,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,sample_set,aim_fraction_colu,aim_fraction_arab,species_gambcolu_arabiensis,species_gambiae_coluzzii,is_arabiensis,is_gamb_colu,is_gambiae,is_coluzzii,site
0,LUA047,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,0.945,0.001,gamb_colu,coluzzii,False,True,False,True,3
1,LUA049,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,0.933,0.001,gamb_colu,coluzzii,False,True,False,True,3
2,LUA051,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,0.937,0.002,gamb_colu,coluzzii,False,True,False,True,3
3,LUA061,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,0.938,0.002,gamb_colu,coluzzii,False,True,False,True,3
4,LUA078,Joao Pinto,Angola,Luanda,2009,4,-8.884,13.302,F,AG1000G-AO,0.926,0.001,gamb_colu,coluzzii,False,True,False,True,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2616,K92,Martin Donnelly,Uganda,Kihihi,2012,11,-0.751,29.701,F,AG1000G-UG,0.026,0.002,gamb_colu,gambiae,False,True,True,False,9
2617,K93,Martin Donnelly,Uganda,Kihihi,2012,11,-0.751,29.701,F,AG1000G-UG,0.029,0.003,gamb_colu,gambiae,False,True,True,False,9
2618,K94,Martin Donnelly,Uganda,Kihihi,2012,11,-0.751,29.701,F,AG1000G-UG,0.026,0.002,gamb_colu,gambiae,False,True,True,False,9
2619,K95,Martin Donnelly,Uganda,Kihihi,2012,11,-0.751,29.701,F,AG1000G-UG,0.029,0.002,gamb_colu,gambiae,False,True,True,False,9
